In [1]:
import yfinance as yf
import pandas as pd
import ta
from backtesting import Backtest, Strategy
from backtesting.lib import crossover

C:\Users\Prateek\anaconda3\lib\site-packages\backtesting\_plotting.py:50: UserWarning: Jupyter Notebook detected. Setting Bokeh output to notebook. This may not work in Jupyter clients without JavaScript support (e.g. PyCharm, Spyder IDE). Reset with `backtesting.set_bokeh_output(notebook=False)`.
  warnings.warn('Jupyter Notebook detected. '


Loading BokehJS ...

In [2]:
df = yf.download('M&M.NS', start = '2021-05-01', interval = '1d')

[*********************100%***********************]  1 of 1 completed


In [3]:
class Trade(Strategy):
    
    def init(self):
        close = self.data.Close
        high = self.data.High
        low = self.data.Low
        self.stoch_k = self.I(ta.momentum.stochrsi_k, pd.Series(close))
        self.stoch_d = self.I(ta.momentum.stochrsi_d, pd.Series(close))
        self.EMA_9 = self.I(ta.trend.ema_indicator, pd.Series(close), window = 9)
        self.EMA_12 = self.I(ta.trend.ema_indicator, pd.Series(close), window = 12)
        self.EMA_26 = self.I(ta.trend.ema_indicator, pd.Series(close), window = 26)
        self.atr = self.I(ta.volatility.average_true_range, pd.Series(high), pd.Series(low), pd.Series(close))
        
    def next(self):
        price = self.data.Close
        if(crossover(self.stoch_k, self.stoch_d) and
          price > self.EMA_9 and
          self.EMA_9 > self.EMA_12 and
          self.EMA_12 > self.EMA_26):
            
            sl = price - self.atr * 2
            tp = price + self.atr * 2
            self.buy(sl = sl, tp = tp)
            

In [4]:
bt = Backtest(df, Trade, cash = 10000, exclusive_orders = True)

In [5]:
output = bt.run()

In [6]:
output

Start                     2021-05-03 00:00:00
End                       2022-08-12 00:00:00
Duration                    466 days 00:00:00
Exposure Time [%]                   23.364486
Equity Final [$]                 12011.577521
Equity Peak [$]                  12011.577521
Return [%]                          20.115775
Buy & Hold Return [%]               67.268734
Return (Ann.) [%]                   15.475475
Volatility (Ann.) [%]               17.599599
Sharpe Ratio                         0.879308
Sortino Ratio                        1.789961
Calmar Ratio                         1.757796
Max. Drawdown [%]                   -8.803908
Avg. Drawdown [%]                   -5.405383
Max. Drawdown Duration      133 days 00:00:00
Avg. Drawdown Duration       40 days 00:00:00
# Trades                                   13
Win Rate [%]                        61.538462
Best Trade [%]                       6.109452
Worst Trade [%]                     -6.375949
Avg. Trade [%]                    

In [7]:
bt.plot()

Row(id='1674', ...)